### Tasks
0. unifying code: data loading, data structures, imports, 

1. make the pie plot about num of keypoints and num of images / annotations for each dataset like the following:

2.  animal / image size analysis on each bird dataset (histogram). Like this below

3. keypoints vs. bounding box analysis (how many boxes look unusual? how many keypoints are occluded vs. not occluded? Try to make histogram and nice analysis)

4. visualize the keypoints and bounding box on 10 images from each bird dataset

5. Run a k-means clustering to detect outliers and visualize outliers vs. centroid keypoints

### TODOs
- methods => into scripts
    - data loading scripts: data.py
    - visualisation/plotting scripts: visualisation.py
    - stats script: stats.py
    - utilities script: utils.py

In [8]:
from src.data import *

In [4]:
root = "/home/dikra/media/dikra/PhD/DATA/DLC24_Data/tiny_all_bird_merged_coco"
images = f"{root}/images"
annotations = f"{root}/annotations"
train_file = f"{annotations}/train.json"

data = load_data(train_file)
data

OSError: [Errno 107] Transport endpoint is not connected: '/home/dikra/media/dikra/PhD/DATA/DLC24_Data/tiny_all_bird_merged_coco/annotations/train.json'